### Configuration of the environment

In [ ]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5
import seqio

BASE_DIR = "gs://bucket_code_completion_python" 
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Installing dependencies...
     |████████████████████████████████| 153kB 6.9MB/s 
     |████████████████████████████████| 368kB 22.5MB/s 
     |████████████████████████████████| 3.9MB 23.3MB/s 
     |████████████████████████████████| 256kB 55.3MB/s 
     |████████████████████████████████| 2.3MB 55.5MB/s 
     |████████████████████████████████| 1.2MB 50.5MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 4.3MB 42.9MB/s 
     |████████████████████████████████| 901kB 59.4MB/s 
     |████████████████████████████████| 3.3MB 2.1MB/s 
Running on TPU: grpc://10.122.106.138:8470


### Loading of the tsv files
We loaded the tsv files, please be sure to upload them on the bucket and to copy the correct path

In [2]:
#Validation(train and test on the same dataset)

nq_tsv_path = {
    "train":      'gs://bucket_code_completion_python/Python_T5/ft_datasets/train.tsv',
    "validation": 'gs://bucket_code_completion_python/Python_T5/ft_datasets/eval.tsv',
}

num_nq_examples = dict(train=2739780, validation=342473)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for HP tuning.

In [3]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = 'gs://bucket_code_completion_python/Python_T5/code.model'
vocab_path = 'gs://bucket_code_completion_python/Python_T5/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

PYTHON data processing

In [4]:
def nq_python(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_python("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b"def handle_tag_attributes ( self , tool_dependencies_config ) : <NEW_LINE> <IND> rdah = RepositoryDependencyAttributeHandler ( self . app , self . unpopulate ) <NEW_LINE> tah = tag_attribute_handler . TagAttributeHandler ( self . app , rdah , self . unpopulate ) <NEW_LINE> altered = False <NEW_LINE> error_message = '' <NEW_LINE> tree , error_message = xml_util . parse_xml ( tool_dependencies_config ) <NEW_LINE> if tree is None : <NEW_LINE> <IND> return False , None , error_message <extra_id_0> <NEW_LINE> altered , new_root , error_message = tah . process_config ( root , skip_actions_tags = False ) <NEW_LINE> return altered , new_root , error_message <NEW_LINE> <DEIND>", 'output': b'<NEW_LINE> <DEIND> root = tree . getroot ( )'}
{'input': b"def handle_tag_attributes ( self , tool_dependencies_config ) : <NEW_LINE> <IND> rdah = RepositoryDependencyAttributeHandler ( self . app , self . unpopulate ) <NEW_LINE> tah <extra_id_0> <NEW_LINE> altered = F

In [5]:
def python_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['PYTHON:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [6]:
t5.data.TaskRegistry.remove('python')
t5.data.TaskRegistry.add(
    "python",
    dataset_fn=nq_python,
    splits=["train", "validation"],
    text_preprocessor=[python_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples
)

In [7]:
nq_task = t5.data.TaskRegistry.get("python")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.7/dist-packages/seqio/preprocessors.py:90: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b"PYTHON:def setUp ( self ) : <NEW_LINE> <IND> self . logit = numpy . random . normal ( <NEW_LINE> size = self . logit_shape ) . astype ( self . dtype ) <NEW_LINE> self . x = numpy . random . randint ( <NEW_LINE> 0 , 2 , size = self . x_shape ) . astype <extra_id_0> <NEW_LINE> self . gy = numpy . random . normal ( size = self . x_shape ) . astype ( self . dtype ) <NEW_LINE> self . ggx = numpy . random . normal ( <NEW_LINE> size = self . logit_shape ) . astype ( self . dtype ) <NEW_LINE> self . backward_options = { 'atol' : 1e-2 , 'rtol' : 1e-2 } <NEW_LINE> <DEIND>", 'inputs': array([  744,     2,   223,     2,  2211,    81,  1204,   315,     9,
          15,    10,    16,     5,     7,     3,     6,     4,     5,
          11,     4,    15,     8,  7702,    14,   821,     8,   370,
           8,  2410,     9,     5,     7,     3,     6,     4,   191,
          14,    15,     8,  7702,     3,   372,    10,     8,  1464,
   

### Hyper Parameter tuning
You can run the HP tuning using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in), *PATH_GIN_FILE* (the gin file configuration for this HP tuning) and *PRETRAINED_DIR* (the folder that contains the pretrained model).  
**Keep attention** to change the *pretrained_model_dir* in finetune step (if you are starting the HP tuning from scratch you have to set the value *PRETRAINED_DIR*, if you are restarting the HP tuning from a previous saved checkpoint you have to set the value *MODEL_DIR*)

In [8]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    ["python"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)

In [9]:
from mesh_tensorflow.transformer.learning_rate_schedules import truncated_rsqrt

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_code_completion_python/Python_T5/pretrained_with_masking'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = truncated_rsqrt,
    sequence_length={"inputs": 256, "targets": 256},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [10]:
PATH_GIN_FILE = 'gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/operative_config.gin'
import gin

with gin.unlock_config():
    gin.parse_config_file(PATH_GIN_FILE)
    #RUN FINE-TUNING
    FINETUNE_STEPS = 50000
    model.finetune(
        mixture_or_task_name="all_tasks",
        pretrained_model_dir=PRETRAINED_DIR,
        finetune_steps=FINETUNE_STEPS
    )

INFO:root:system_path_file_exists:gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/operative_config.gin
ERROR:root:Path not found: gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/operative_config.gin
INFO:root:system_path_file_exists:gs://bucket_code_completion_python/Python_T5/pretrained_with_masking/operative_config.gin
ERROR:root:Path not found: gs://bucket_code_completion_python/Python_T5/pretrained_with_masking/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.90.19.178:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.90.19.178:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.90.19.178:8470', '_evaluation_master': 'grpc

/usr/local/lib/python3.7/dist-packages/seqio/preprocessors.py:90: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('d_ff', 'model'), ('batch', 'batch'), ('experts', 'batch'), ('heads', 'model'), ('vocab', 'model'), ('ensemble', 'ensemble')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7fd88258a910>
INF

### Evaluate the performances
#### We use this cell only to create the inputs and targets, then with predict we can do all the things faster

In [11]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = 512
model.eval(
    mixture_or_task_name="all_tasks",
    # mixture_or_task_name="all_tasks",
    checkpoint_steps=-1 #evaluate only last checkpoint
)

INFO:root:system_path_file_exists:gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model/operative_config.gin
ERROR:root:Path not found: gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model/operative_config.gin
INFO:absl:Adding task 'python' with predict metric_fn(s).
/usr/local/lib/python3.7/dist-packages/seqio/preprocessors.py:90: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
INFO:absl:Skipping packing/padding for 'python' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 3197, 'targets': 816}
INFO:absl:Evaluating checkpoint step: 450000


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.90.19.178:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.90.19.178:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.90.19.178:8470', '_evaluation_master': 'grpc

INFO:absl:Padding 'python' with sequence lengths: {'inputs': 3197, 'targets': 816}


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('d_ff', 'model'), ('batch', 'batch'), ('experts', 'batch'), ('heads', 'model'), ('vocab', 'model'), ('ensemble', 'ensemble')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7fd962d31150>
INF

ResourceExhaustedError: ignored

In [12]:
# we used model.predict function (setting beam_size)

vocabulary_predict=get_default_vocabulary()

model.predict(input_file='gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model/validation_eval/python_inputs', output_file='gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model/validation_eval/python_predictions',
              checkpoint_steps=-1, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)

INFO:root:system_path_file_exists:gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model/operative_config.gin
ERROR:root:Path not found: gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_code_completion_python/Python_T5/HP_TUNING/ISR/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.90.19.178:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.90.19.178:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.90.19.178:8470', '_evaluation_master': 'grpc